In [ ]:
import numpy as np
from scipy.integrate import solve_ivp

def rossler(xyz, a=0.2, b=0.2, c=5.7):
    x, y, z = xyz
    x_dot = -y - z
    y_dot = x + a * y
    z_dot = b + z * (x - c)
    return np.array([x_dot, y_dot, z_dot])

def emb_dm_y(state, a=0.2, b=0.2, c=5.7):
    x, y, z = state
    x_dot = y
    y_dot = x + a * y
    z_dot = -y - z + a * x + a * a * y
    return np.array([x_dot, y_dot, z_dot])

def emb_dm_x(state, a=0.2, b=0.2, c=5.7):
    x, y, z = state
    x_dot = x
    y_dot = -y - z
    z_dot = -x - a * y - b - z * (x - c)
    return np.array([x_dot, y_dot, z_dot])

def emb_dm_z(state, a=0.2, b=0.2, c=5.7):
    x, y, z = state
    x_dot = z
    y_dot = b + z * (x - c)
    z_dot = (b + z * (x - c)) * (x - c) + z * (-y - z)
    return np.array([x_dot, y_dot, z_dot])

def emb_dm_yz(state, a=0.2, b=0.2, c=5.7):
    x, y, z = state
    x_dot = y + z
    y_dot = b + x + a * y - c * z + x * z
    z_dot = (
        -b * c
        + (a + b) * x
        + (a * a - 1) * y
        + (c * c - 1) * z
        - (2 * c + 1) * x * z
        - z * z
        + x * x * z
    )
    return np.array([x_dot, y_dot, z_dot])

def emb_dm_zx(state, a=0.2, b=0.2, c=5.7):
    x, y, z = state
    x_dot = z + x
    y_dot = -y - z + b + z * (x - c)
    z_dot = (
        -b * (c + 1)
        + (b - 1) * x
        - a * y
        + c * (c + 1) * z
        + (1 - 2 * c) * x * z
        - y * z
        - z * z
    )
    return np.array([x_dot, y_dot, z_dot])

def emb_dm_xy(state, a=0.2, b=0.2, c=5.7):
    x, y, z = state
    x_dot = x + y
    y_dot = x + (a - 1) * y - z
    z_dot = -b + (a - 1) * x + (a * a - a + 1) * y + (c - 1) * z - x * z
    return np.array([x_dot, y_dot, z_dot])

def emb_y_z_y(state, a=0.2, b=0.2, c=5.7):
    x, y, z = state
    x_dot = y
    y_dot = z
    z_dot = x + a * y
    return np.array([x_dot, y_dot, z_dot])

def emb_x_y_x(state, a=0.2, b=0.2, c=5.7):
    x, y, z = state
    x_dot = x
    y_dot = y
    z_dot = -y - z
    return np.array([x_dot, y_dot, z_dot])

def emb_x_y_y(state, a=0.2, b=0.2, c=5.7):
    x, y, z = state
    x_dot = x
    y_dot = y
    z_dot = x + a * y
    return np.array([x_dot, y_dot, z_dot])

def emb_y_z_z(state, a=0.2, b=0.2, c=5.7):
    x, y, z = state
    x_dot = y
    y_dot = z
    z_dot = b + z * (x - c)
    return np.array([x_dot, y_dot, z_dot])

def emb_x_z_z(state, a=0.2, b=0.2, c=5.7):
    x, y, z = state
    x_dot = x
    y_dot = z
    z_dot = b + z * (x - c)
    return np.array([x_dot, y_dot, z_dot])

def emb_x_z_x(state, a=0.2, b=0.2, c=5.7):
    x, y, z = state
    x_dot = x
    y_dot = z
    z_dot = -y - z
    return np.array([x_dot, y_dot, z_dot])

def vizinhos(x, v, epsilon):
    dist = np.sum(np.abs(x - v), axis=1)   # L1
    iv = np.where(dist < epsilon)[0]
    xv = x[iv, :]
    return xv, iv

def svdo(X, p, epsilon):
    S1p_list = []
    N, c = X.shape

    step = int(p)
    if step < 1:
        step = 1

    for j in range(0, N, step):
        center_vector = X[j, :]
        xv, iv = vizinhos(X, center_vector, epsilon)

        if len(iv) > c:
            U, S, Vh = np.linalg.svd(xv, full_matrices=False)
            val = S[0] * 100.0 / np.sum(S)
            S1p_list.append(val)

    if len(S1p_list) == 0:
        return np.array([]), 0.0, 0

    S1p_arr = np.array(S1p_list)
    mean_val = np.mean(S1p_arr)
    std_val = np.std(S1p_arr, ddof=1)

    if std_val == 0:
        svdox_val = 0.0
    else:
        svdox_val = (mean_val / 100.0) / std_val

    return S1p_arr, svdox_val, len(S1p_list)

def integrate_rossler(t0=0.0, tf=200.0, dt=0.01,
                      x0=None, discard=0, decim=1):
    if x0 is None:
        x0 = np.random.randn(3)

    t_eval = np.arange(t0, tf, dt)
    sol = solve_ivp(lambda t, x: rossler(x),
                    [t0, tf],
                    x0,
                    t_eval=t_eval,
                    rtol=1e-7, atol=1e-9)
    X_full = sol.y.T
    X = X_full
    return X

def run_svdo_for_embedding(emb_func, name,
                           num_iterations=10,
                           N=10000,
                           noise_level=0.0,
                           p=10):
    vals = []

    for k in range(num_iterations):
        X_base = integrate_rossler()
        if X_base.shape[0] < N:
            continue
        X_base = X_base[:N, :]
        Y = np.array([emb_func(state) for state in X_base])
        s = Y[:, 0]
        delta = np.max(s) - np.min(s)
        epsilon = 0.1 * delta

        if noise_level > 0:
            std_s = np.std(s, ddof=1)
            noise = np.random.randn(*s.shape) * noise_level * std_s
            Y_noisy = Y.copy()
            Y_noisy[:, 0] = s + noise
        else:
            Y_noisy = Y

        _, svdo_val, count = svdo(Y_noisy, p=p, epsilon=epsilon)
        if count > 0:
            vals.append(svdo_val)

    vals = np.array(vals)
    if len(vals) == 0:
        print(f"{name:12s}: no valid neighborhoods")
        return None

    mean_v = np.mean(vals)
    std_v = np.std(vals, ddof=1)
    print(f"{name:12s}: SVDO = {mean_v:.4f} ± {std_v:.4f}  (n={len(vals)})")
    return mean_v, std_v

embeddings = {
    "dm_y":   emb_dm_y,
    "dm_x":   emb_dm_x,
    "dm_z":   emb_dm_z,
    "dm_yz":  emb_dm_yz,
    "dm_zx":  emb_dm_zx,
    "dm_xy":  emb_dm_xy,
    "y_z_y":  emb_y_z_y,
    "x_y_x":  emb_x_y_x,
    "x_y_y":  emb_x_y_y,
    "y_z_z":  emb_y_z_z,
    "x_z_z":  emb_x_z_z,
    "x_z_x":  emb_x_z_x,
}

print("Running SVDO for Rössler embeddings...\n")
for name, emb_func in embeddings.items():
    run_svdo_for_embedding(emb_func, name,
                            num_iterations=10,
                            N=10000,
                            noise_level=0.0,
                            p=10)

Running SVDO for Rössler embeddings...

dm_y        : SVDO = 0.1776 ± 0.0742  (n=10)
dm_x        : SVDO = 0.1588 ± 0.0305  (n=10)
dm_z        : SVDO = 0.0672 ± 0.0054  (n=10)
dm_yz       : SVDO = 0.0959 ± 0.0301  (n=10)
dm_zx       : SVDO = 0.1041 ± 0.0176  (n=10)
dm_xy       : SVDO = 0.1859 ± 0.0630  (n=10)
y_z_y       : SVDO = 0.1380 ± 0.0449  (n=10)
x_y_x       : SVDO = 0.1478 ± 0.0517  (n=10)
x_y_y       : SVDO = 0.1590 ± 0.0501  (n=10)
y_z_z       : SVDO = 0.1782 ± 0.0286  (n=10)
x_z_z       : SVDO = 0.1923 ± 0.0083  (n=10)
x_z_x       : SVDO = 0.1067 ± 0.0361  (n=10)
